In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import cv2
import skimage.morphology as morp
from skimage.filters import rank

import tensorflow as tf
import dotscience as ds

In [2]:
ds.interactive()

In [3]:
# Show current TensorFlow version
tf.__version__


'1.12.0'

In [4]:
%%bash
DATAFILE="https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip"
if [ -d "/tmp/traffic-signs-data" ]; then
    echo "Data already downloaded"
else
    echo "Downloading data from $DATAFILE"
    mkdir /tmp/traffic-signs-data
    curl -s -o /tmp/traffic-signs-data/traffic-signs-data.zip $DATAFILE
    (cd /tmp/traffic-signs-data && unzip traffic-signs-data.zip && rm -f traffic-signs-data.zip)
fi


Data already downloaded


In [5]:
training_file = "/tmp/traffic-signs-data/train.p"
validation_file= "/tmp/traffic-signs-data/valid.p"
testing_file = "/tmp/traffic-signs-data/test.p"

ds.input(training_file)
ds.input(validation_file)
ds.input(testing_file)

train = pickle.load(open(training_file,"rb"))
valid = pickle.load(open(validation_file,"rb"))
test = pickle.load(open(testing_file,"rb"))




In [6]:
import csv
# Mapping ClassID to traffic sign names
signs = []
with open('signnames.csv', 'r') as csvfile:
    signnames = csv.reader(csvfile, delimiter=',')
    next(signnames,None)
    for row in signnames:
        signs.append(row[1])
    csvfile.close()
    
ds.input("signnames.csv")

'signnames.csv'

In [7]:
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

# Number of training examples
n_train = X_train.shape[0]

# Number of testing examples
n_test = X_test.shape[0]

# Number of validation examples.
n_validation = X_valid.shape[0]

# What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_train))

print("Number of training examples: ", n_train)
print("Number of testing examples: ", n_test)
print("Number of validation examples: ", n_validation)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples:  34799
Number of testing examples:  12630
Number of validation examples:  4410
Image data shape = (32, 32, 3)
Number of classes = 43


In [8]:
X_train[0].shape

(32, 32, 3)

# preprocess data

In [9]:
# define helper functions
def list_images(dataset, dataset_y, ylabel="", cmap=None):
    """
    Display a list of images in a single figure with matplotlib.
        Parameters:
            images: An np.array compatible with plt.imshow.
            lanel (Default = No label): A string to be used as a label for each image.
            cmap (Default = None): Used to display gray images.
    """
    plt.figure(figsize=(15, 16))
    for i in range(6):
        plt.subplot(1, 6, i+1)
        indx = random.randint(0, len(dataset))
        #Use gray scale color map if there is only one channel
        cmap = 'gray' if len(dataset[indx].shape) == 2 else cmap
        plt.imshow(dataset[indx], cmap = cmap)
        plt.xlabel(signs[dataset_y[indx]])
        plt.ylabel(ylabel)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.show()
    
def histogram_plot(dataset, label):
    """
    Plots a histogram of the input data.
        Parameters:
            dataset: Input data to be plotted as a histogram.
            lanel: A string to be used as a label for the histogram.
    """
    hist, bins = np.histogram(dataset, bins=n_classes)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.bar(center, hist, align='center', width=width)
    plt.xlabel(label)
    plt.ylabel("Image count")
    plt.show()
    
def gray_scale(image):
    """
    Convert images to gray scale.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

def local_histo_equalize(image):
    """
    Apply local histogram equalization to grayscale images.
        Parameters:
            image: A grayscale image.
    """
    kernel = morp.disk(30)
    img_local = rank.equalize(image, selem=kernel)
    return img_local

def image_normalize(image):
    """
    Normalize images to [0, 1] scale.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    image = np.divide(image, 255)
    return image

def preprocess(data):
    """
    Applying the preprocessing steps to the input data.
        Parameters:
            data: An np.array compatible with plt.imshow.
    """
    gray_images = list(map(gray_scale, data))
    equalized_images = list(map(local_histo_equalize, gray_images))
    n_training = data.shape
    normalized_images = np.zeros((n_training[0], n_training[1], n_training[2]))
    for i, img in enumerate(equalized_images):
        normalized_images[i] = image_normalize(img)
    normalized_images = normalized_images[..., None]
    return normalized_images

In [10]:
X_valid_preprocessed = preprocess(X_valid)
X_test_preprocessed = preprocess(X_test)
X_train_preprocessed = preprocess(X_train)

# make model

In [11]:
num_classes=43
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(32, 32, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(tf.keras.layers.Conv2D(32, (5, 5), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1000, activation='relu'))
model.add(tf.keras.layers.Dense(units = num_classes, activation='softmax'))

model.compile(optimizer=ds.parameter("optimizer", 'adam'),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_preprocessed, y_train,
          epochs=ds.parameter("epochs", 10),
          verbose=1,
          validation_data=(X_valid_preprocessed, y_valid))

Train on 34799 samples, validate on 4410 samples
Epoch 1/10
34799/34799 [==============================] - 7s 208us/step - loss: 0.5985 - acc: 0.8336 - val_loss: 0.3754 - val_acc: 0.8864
Epoch 2/10
34799/34799 [==============================] - 4s 127us/step - loss: 0.0950 - acc: 0.9717 - val_loss: 0.2516 - val_acc: 0.9270
Epoch 3/10
34799/34799 [==============================] - 4s 127us/step - loss: 0.0489 - acc: 0.9854 - val_loss: 0.3513 - val_acc: 0.9188
Epoch 4/10
34799/34799 [==============================] - 5s 130us/step - loss: 0.0339 - acc: 0.9898 - val_loss: 0.3332 - val_acc: 0.9227
Epoch 5/10
34799/34799 [==============================] - 4s 128us/step - loss: 0.0301 - acc: 0.9910 - val_loss: 0.2395 - val_acc: 0.9438
Epoch 6/10
34799/34799 [==============================] - 4s 127us/step - loss: 0.0249 - acc: 0.9928 - val_loss: 0.3589 - val_acc: 0.9281
Epoch 7/10
34799/34799 [==============================] - 4s 127us/step - loss: 0.0211 - acc: 0.9937 - val_loss: 0.2230 - v

In [12]:
model.evaluate(X_test_preprocessed, y_test)

12630/12630 [==============================] - 1s 46us/step


[0.43626571019851007, 0.9296912113919866]

In [18]:
ds.summary("accuracy", model.evaluate(X_test_preprocessed, y_test)[1])

12630/12630 [==============================] - 1s 46us/step


0.9296912113919866

In [19]:
model.predict(X_test_preprocessed)

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 5.2460611e-24,
        1.2476710e-31, 2.1871050e-35],
       [1.4405721e-12, 9.9999321e-01, 4.2519357e-09, ..., 4.8632153e-12,
        3.1908508e-21, 5.0349353e-11],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [7.9952991e-07, 7.9981948e-04, 1.4570546e-04, ..., 3.1646925e-07,
        1.0466221e-02, 6.7897752e-05],
       [3.6614904e-12, 2.8997500e-08, 4.7649964e-04, ..., 6.0561909e-03,
        3.6400119e-14, 3.4698910e-08],
       [2.0499307e-13, 3.1294252e-11, 1.1283398e-06, ..., 3.4787002e-04,
        9.3975541e-14, 1.0938111e-10]], dtype=float32)

In [20]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = "./model"
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))
if os.path.isdir(export_path):
  print('\nAlready saved a model, cleaning up\n')
  !rm -r {export_path}

tf.saved_model.simple_save(
    tf.keras.backend.get_session(),
    export_path,
    inputs={'input_image': model.input},
    outputs={t.name:t for t in model.outputs})

print('\nSaved model:')
!ls -l {export_path}

export_path = ./model/1


Already saved a model, cleaning up

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./model/1/saved_model.pb

Saved model:
total 1
-rw-r--r-- 1 root root 195414 Mar 13 02:29 saved_model.pb
drwxr-xr-x 2 root root      4 Mar 13 02:29 variables


In [21]:
!saved_model_cli show --dir {export_path} --all



MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_image'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 32, 32, 1)
        name: conv2d_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1/Softmax:0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 43)
        name: dense_1/Softmax:0
  Method name is: tensorflow/serving/predict


In [22]:
ds.label("model.directory", ds.output("model"))
ds.label("model.framework", "tensorflow")
ds.label("model.framework.version", tf.__version__)

'1.12.0'

In [23]:
ds.publish()

[[DOTSCIENCE-RUN:ea1db22b-dc0a-414c-91d9-2b676d15fbe7]]{
    "end": "20190313T023001.299301",
    "input": [],
    "labels": {
        "model.directory": "model",
        "model.framework": "tensorflow",
        "model.framework.version": "1.12.0"
    },
    "output": [
        "model"
    ],
    "parameters": {},
    "start": "20190313T022849.613617",
    "summary": {
        "accuracy": "0.9296912113919866"
    },
    "version": "1"
}[[/DOTSCIENCE-RUN:ea1db22b-dc0a-414c-91d9-2b676d15fbe7]]
